In [1]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
import pandas as pd
import numpy as np
import math
import os
import json
import joblib

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from funcionesComunes import *

2024-12-17 11:43:22.702556: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-17 11:43:22.705555: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-17 11:43:22.715118: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-17 11:43:22.730222: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-17 11:43:22.734869: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-17 11:43:22.746961: I tensorflow/core/platform/cpu_feature_gu

In [ ]:
for archivo in os.listdir("RCPMergedMLPandas/"):
    if "_X_test" in archivo:
        print(archivo)

        # Leemos información ANN
        with open(f'models/ANN/{archivo.split("_X_test")[0]}_merged_best_models_updated_no_train.json', 'r') as f:
            ANN_models = json.load(f)

        # Leemos información GBT
        with open(f'models/GBT/{archivo.split("_X_test")[0]}_merged_best_models_updated_no_train.json', 'r') as f:
            GBT_models = json.load(f)

        # Leemos información LR
        with open(f'models/LR/{archivo.split("_X_test")[0]}_merged_best_models_updated_no_train.json', 'r') as f:
            LR_models = json.load(f)

        # Leemos información RF
        with open(f'models/RF/{archivo.split("_X_test")[0]}_merged_best_models_updated_no_train.json', 'r') as f:
            RF_models = json.load(f)

        # Leemos información SVM
        with open(f'models/SVM/{archivo.split("_X_test")[0]}_merged_best_models_updated_no_train.json', 'r') as f:
            SVM_models = json.load(f)


        # Extraer la lista de configuraciones de parámetros
        ANN_params = ANN_models[0]["params"]
        GBT_params = GBT_models[0]["params"]
        LR_params = LR_models[0]["params"]
        RF_params = RF_models[0]["params"]
        SVM_params = SVM_models[0]["params"]

        # Creamos el modelo
        ANN = MLPRegressor(**ANN_params)
        GBT = GradientBoostingRegressor(**GBT_params)
        LR = Ridge(**LR_params)
        RF = RandomForestRegressor(**RF_params)
        SVM = SVR(**SVM_params)

        # Listado de modelos
        models = [ANN, GBT, LR, RF, SVM]
        modelsName = ["ANN", "GBT", "LR", "RF", "SVM"]
        
        # Para cada modelo
        for model, name in zip(models, modelsName): 

            # Creamos un df que contiene los resultados de los modelos freeze de cada especie
            df_Freeze = pd.DataFrame(columns = ["File", "TrainMSE", "TrainRMSE", "TrainR2", "TrainMAPE", 
                                                "ValidationMSE", "ValidationRMSE", "ValidationR2", "ValidationMAPE",
                                                "TestMSE", "TestRMSE", "TestR2", "TestMAPE"])

            # Entrenamos el modelo con el resto de archivos
            for subArchivo in os.listdir("RCPMergedMLPandas/"):
                
                if subArchivo.split("_X_test")[0] != archivo.split("_X_test")[0] and "_X_test" in subArchivo:
                    print("#"*40)
                    print(f'SUBARCHIVO {subArchivo.split("_X_test")[0]}')

                    # Leemos datos
                    X_train = pd.read_csv(f'RCPMergedMLPandas/{subArchivo.split("_X_test")[0]}_X_train.csv')
                    y_train = pd.read_csv(f'RCPMergedMLPandas/{subArchivo.split("_X_test")[0]}_y_train.csv')
                    
                    X_test = pd.read_csv(f'RCPMergedMLPandas/{subArchivo.split("_X_test")[0]}_X_test.csv')
                    y_test = pd.read_csv(f'RCPMergedMLPandas/{subArchivo.split("_X_test")[0]}_y_test.csv')
                    
                    X_val = pd.read_csv(f'RCPMergedMLPandas/{subArchivo.split("_X_test")[0]}_X_val.csv')
                    y_val = pd.read_csv(f'RCPMergedMLPandas/{subArchivo.split("_X_test")[0]}_y_val.csv')

                    valorNormalizacion = np.load(f'RCPMergedMLPandas/{subArchivo.split("_X_test")[0]}_valorNormalizacion.npz', allow_pickle=True)
                    valorNormalizacion = valorNormalizacion['valorNormalizacion'].item()

                    # Crear una nueva instancia del modelo en cada iteración
                    model = model.__class__(**model.get_params())

                    # Hacemos el fit
                    model.fit(X_train, y_train.values.ravel())

                    # Predecir en el conjunto de entrenamiento
                    y_train_pred = model.predict(X_train)
                    y_train_true_denorm = y_train.values.ravel() * np.array([valorNormalizacion[nt] for nt in X_train["nametag_encoded"].values])
                    y_train_pred_denorm = y_train_pred.ravel() * np.array([valorNormalizacion[nt] for nt in X_train["nametag_encoded"].values])

                    # Calcular métricas en el conjunto de entrenamiento
                    train_mse = mean_squared_error(y_train_true_denorm, y_train_pred_denorm)
                    train_rmse = np.sqrt(train_mse)
                    train_r2 = r2_score(y_train_true_denorm, y_train_pred_denorm)
                    train_mape = mean_absolute_percentage_error(y_train_true_denorm, y_train_pred_denorm) * 100
                    
                    # Predecir en el conjunto de validación
                    y_val_pred = model.predict(X_val)
                    y_val_true_denorm = y_val.values.ravel() * np.array([valorNormalizacion[nt] for nt in X_val["nametag_encoded"].values])
                    y_val_pred_denorm = y_val_pred.ravel() * np.array([valorNormalizacion[nt] for nt in X_val["nametag_encoded"].values])
                    
                    # Calcular métricas en el conjunto de validación
                    val_mse = mean_squared_error(y_val_true_denorm, y_val_pred_denorm)
                    val_rmse = np.sqrt(val_mse)
                    val_r2 = r2_score(y_val_true_denorm, y_val_pred_denorm)
                    val_mape = mean_absolute_percentage_error(y_val_true_denorm, y_val_pred_denorm) * 100
                    
                    # Predecir en el conjunto de prueba
                    y_test_pred = model.predict(X_test)
                    y_test_true_denorm = y_test.values.ravel() * np.array([valorNormalizacion[nt] for nt in X_test["nametag_encoded"].values])
                    y_test_pred_denorm = y_test_pred.ravel() * np.array([valorNormalizacion[nt] for nt in X_test["nametag_encoded"].values])

                    # Calcular métricas en el conjunto de prueba
                    test_mse = mean_squared_error(y_test_true_denorm, y_test_pred_denorm)
                    test_rmse = np.sqrt(test_mse)
                    test_r2 = r2_score(y_test_true_denorm, y_test_pred_denorm)
                    test_mape = mean_absolute_percentage_error(y_test_true_denorm, y_test_pred_denorm) * 100

                    print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Train): {train_mse}, {train_rmse}, {train_r2}, {train_mape}")
                    print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Val): {val_mse}, {val_rmse}, {val_r2}, {val_mape}")
                    print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Test): {test_mse}, {test_rmse}, {test_r2}, {test_mape}")

                    # Guardamos los datos calculados
                    df_Freeze.loc[len(df_Freeze)] = [subArchivo.split("_X_test")[0], train_mse, train_rmse, train_r2, train_mape, 
                                                    val_mse, val_rmse, val_r2, val_mape, 
                                                    test_mse, test_rmse, test_r2, test_mape]
            
            # Guardamos el archivo
            df_Freeze.to_csv(f'resultsTransfer/{name}/resultadosTrasnfer_{archivo.split("_X_test")[0]}.csv', index=False)

Pinus gerardiana_X_test.csv
########################################
SUBARCHIVO Tsuga dumosa
RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 757.5305829867795, 27.523273478763013, -1.098886440068481, 150.80386744067846
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 1029.8125944098683, 32.09069326782873, -0.8486752836419047, 178.43000516552868
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 1115.8975094623509, 33.40505215476173, -1.4513367281572216, 142.7807602411684
########################################
SUBARCHIVO Juniperus spp. 
RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 763.0184092723732, 27.622787862060072, -10.41219374271519, 625.0869843890899
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 192.88366612208875, 13.888256410438595, -9.020542603491913, 511.99164435183843
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 1146.311817758834, 33.85722696498982, -11.344236487453658, 635.6107677776057
########################################
SUBARCHIVO Juniperus spp. L.
RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 47.127

/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 35.03583627229822, 5.919107726025792, 0.47598151418285506, 110.17189380218866
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 6.54345906368752, 2.5580185815758885, 0.6600592904513587, 79.1750285892399
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 52.362139408850624, 7.236168835015572, 0.43613018566309925, 97.51026112238537
########################################
SUBARCHIVO Juniperus spp. L.
RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 4.383745005323067, 2.093739478856686, 0.645679862915984, 122.50146138903479
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 1.429822259885612, 1.1957517551254575, 0.35519188624628806, 191.44120162459092
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 14.310340100296061, 3.7829010164549723, 0.2307958463687897, 78.13781594602945
########################################
SUBARCHIVO Cedrus deodara


/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 220.7019827918359, 14.856041962509257, 0.4475418213746646, 89.99648942081542
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 234.31095694039368, 15.307219111922116, 0.4589635508941122, 70.86733280525749
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 228.93261095284296, 15.130519189797916, 0.47437719193394445, 68.56256395875863
########################################
SUBARCHIVO Abies spectabilis


...

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 178.61070408871032, 13.364531570119109, 0.3134140656921577, 116.51171827586542
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 165.0758125529795, 12.848183239391455, 0.3761957397644271, 86.31898533426607
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 156.83036913316536, 12.523193248256028, 0.29529329654021474, 66.49386259001339
########################################
SUBARCHIVO Picea smithiana


/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 201.37977341867523, 14.190834133999143, 0.40276051712387406, 68.0325352178486
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 234.5714803235488, 15.315726568581354, 0.30241324132129155, 103.40240637604785
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 208.6330185429078, 14.444134399226137, 0.29218659518902135, 75.08387691810492
########################################
SUBARCHIVO Juniperus turkestanica Komar.


/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 7.12416674832028, 2.6691134761040565, 0.46370824424549506, 64.30523060214868
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 16.287375834909582, 4.0357621132705015, 0.8368921278241155, 239.43621023866996
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 5.002522856191875, 2.2366320341513206, 0.855010012465616, 94.36877951633979
########################################
SUBARCHIVO Tsuga dumosa
RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 204.75200920685072, 14.309158228451132, 0.43269456659212413, 78.14012228695222
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 359.0274132327356, 18.948018715230774, 0.3554894273034279, 90.49709605753364
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 356.60408691920037, 18.88396375020881, 0.21663352748481124, 79.39749230704233
########################################
SUBARCHIVO Juniperus spp. 
RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 27.764984828184208, 5.269248981418909, 0.5847290415640846, 93.47145597478546
RESULTADOS DE MSE, RMSE, R2, 